# Imports

In [1]:
import pandas as pd

In [2]:
import xml.etree.ElementTree as et

In [3]:
import matplotlib.pyplot as plt

In [4]:
import os

In [5]:
import math

In [6]:
import shutil

# Functions

In [7]:
def transform_xml(xml_doc):
    attr = xml_doc.attrib
    for xml in xml_doc.iter('vehicle'):
        _dict = attr.copy()
        _dict.update(xml.attrib)
        
        yield _dict


In [8]:
price_per_liter = 624 / 1.27
eta = 0.45
m = 18000
g = 9.81
mg_to_liter = 0.0000011765
liter_to_joule = 38000000

In [9]:
def get_price_up(emission, delta_h):
    emission = emission * mg_to_liter * liter_to_joule
    emission = emission + (1/eta) * m * g * delta_h
    emission = emission * (1/liter_to_joule)
    emission = emission * price_per_liter
    return emission

In [10]:
def get_price_down(emission, delta_h):
    emission = emission * mg_to_liter * liter_to_joule
    emission = emission - (1/eta) * m * g * delta_h
    emission = emission * (1/liter_to_joule)
    emission = emission * price_per_liter
    return emission

In [11]:
def get_last_folder(path):
    # Normalize the path to handle different separators and remove trailing separator
    normalized_path = os.path.normpath(path)
    # Split the path into components
    folders = normalized_path.split(os.sep)
    # Get the last folder
    last_folder = folders[-1]
    return last_folder

In [12]:
def calc_elevation_up(group):
    z_diff = pd.to_numeric(group['z']).diff()

    # Filter out negative differences (upward movement)
    up = z_diff.apply(lambda x: x if x > 0 else 0)

    # Sum the positive differences to get the total upward movement
    total_up = up.sum()
    return total_up

In [13]:
def calc_elevation_down(group):
    z_diff = pd.to_numeric(group['z']).diff()

    # Filter out negative differences (upward movement)
    down = z_diff.apply(lambda x: x if x < 0 else 0)

    # Sum the positive differences to get the total upward movement
    total_down = down.sum()
    return total_down

In [14]:
def transform_xml_tripinfo(xml_doc):
    attr = xml_doc.attrib
    for xml in xml_doc.iter('tripinfo'):
        _dict = attr.copy()
        _dict.update(xml.attrib)
        
        yield _dict

In [15]:
def get_routeids_df(base_folder):
    file_path = os.path.join(base_folder, "trips.txt")
    return pd.read_csv(file_path, sep=",")

In [16]:
def get_route_id(trip_id, trip_ids):
    trip_id = trip_id[:-2]
    line = trip_ids.loc[trip_ids['trip_id'] == trip_id]
    route_id = line.iloc[0]['route_id']

    return route_id

In [17]:
def transform_xml_stops(xml_doc):
    for route in xml_doc.iter('route'):
        route_dict = route.attrib.copy()
        stops = []
        
        # Iterate over each <stop> element within the current <route> element
        for stop in route.findall('stop'):
            stop_dict = stop.attrib.copy()
            stops.append(stop_dict)
        
        # Include stops in the route dictionary
        route_dict['stops'] = stops
        
        yield route_dict

In [18]:
def main(seed, scale):
    print(seed + " " + scale)

# XML to df

In [19]:
base_folder = "C:\\Users\\Admin\\Sumo\\nap_gellert_e"

In [30]:
file_path = os.path.join(base_folder, "emission.out.xml")
shutil.copyfile(file_path, "output\\emission_")
emission_output = et.parse(file_path)

transform = transform_xml(emission_output.getroot())
emission_output_list = list(transform)

emission_output_df = pd.DataFrame(emission_output_list)
emission_output_df = emission_output_df.drop(emission_output_df.columns[0], axis=1)

#emission_output_df.shape

In [ ]:
#emission_output_df

In [ ]:
file_path = os.path.join(base_folder, "tripinfo.xml")

b_tripinfo_output = et.parse(file_path)

transform = transform_xml_tripinfo(b_tripinfo_output.getroot())
b_tripinfo_output_list = list(transform)

b_tripinfo_output_pd = pd.DataFrame(b_tripinfo_output_list)
b_tripinfo_output_pd = b_tripinfo_output_pd.drop(b_tripinfo_output_pd.columns[0], axis=1)
#b_tripinfo_output_pd

In [ ]:
file_path = os.path.join(base_folder, "gtfs_pt_vehicles.add.xml")
stops = et.parse(file_path)

transform = transform_xml_stops(stops.getroot())
stops_list = list(transform)

stops_pd = pd.DataFrame(stops_list)

In [ ]:
file_path = os.path.join(base_folder, "gtfs_pt_vehicles.add.xml")
vehicles = et.parse(file_path)

transform = transform_xml(vehicles.getroot())
vehicles_list = list(transform)

vehicles_pd = pd.DataFrame(vehicles_list)
vehicles_pd = vehicles_pd.drop(vehicles_pd.columns[0], axis=1)
#vehicles_pd

In [ ]:
route_ids_df = get_routeids_df(base_folder)

# Group by id

In [ ]:
grouped_df = emission_output_df.groupby('id')

list_of_dfs = [group_data for _, group_data in grouped_df]

In [ ]:

results = []
for group_id, group_data in grouped_df:
    avg_speed = group_data['speed'].astype(float).mean()

    time_loss = b_tripinfo_output_pd.loc[b_tripinfo_output_pd['id'] == group_id, 'timeLoss'].values[0]
    route_length = b_tripinfo_output_pd.loc[b_tripinfo_output_pd['id'] == group_id, 'routeLength'].values[0]
    
    route = vehicles_pd.loc[vehicles_pd['id'] == group_id, 'route'].values[0]
    count_stops = stops_pd[stops_pd['id'] == route]['stops'].apply(len).sum()
    
    z_up = calc_elevation_up(group_data)
    z_down = calc_elevation_down(group_data)
    
    d_height = z_up + z_down
    
    fuel_sum = group_data['fuel'].astype(float).sum()
    
    route_id = get_route_id(group_id, route_ids_df)
    
    if(d_height >= 0):
        fuel_sum = get_price_up(group_data['fuel'].astype(float).sum(), d_height)
    else:
        fuel_sum = get_price_down(group_data['fuel'].astype(float).sum(), d_height)
    
    # Store the results in a dictionary
    group_result = {
        'routeid': route_id,
        'id': group_id,
        'avgSpeed': avg_speed,
        'fuel': fuel_sum,
        'timeloss': time_loss,
        'routeLength': route_length,
        'numOfStops': count_stops,
        'up': z_up,
        'down': z_down
    }
    
    # Append the dictionary to the results list
    results.append(group_result)

# Convert the results list to a DataFrame
result_df = pd.DataFrame(results)
#print(result_df)


# CSV

In [ ]:
tableEmission = pd.read_csv('emissionData.csv', delimiter=';')

#tableEmission.shape

## settings

In [ ]:
locSetting = get_last_folder(base_folder)
seedSetting = 'dd'
trafficScaleSetting = 'wh'

In [ ]:
for index, row in result_df.iterrows():
    row_data = {
        'routeId': row['routeid'],
        'loc': locSetting,
        'tripId': row['id'],
        'seed': seedSetting,
        'avgSpeed': row['avgSpeed'],
        'timeloss': row['timeloss'],
        'route_length': row['routeLength'],
        'elevation_up': row['up'],
        'elevation_down': row['down'],
        'trafficScale': trafficScaleSetting,
        'numOfStops': row['numOfStops'],
        'emission': row['fuel']
    }
    temp_df = pd.DataFrame([row_data])
    #any empty or all-NA columns in tableEmission are excluded before concatenating the DataFrames
    tableEmission = tableEmission.dropna(axis=1, how='all')

    tableEmission = pd.concat([tableEmission, temp_df], ignore_index=True)


In [ ]:
tableEmission.to_csv('batteryData.csv', index=False, sep=';')
#tableEmission

,routeId,loc,tripId,seed,avgSpeed,timeloss,route_length,elevation_up,elevation_down,trafficScale,numOfStops,emission
0,0330,nap_gellert_e,C622152254.0,dd,4.476667,107.76,1023.22,18.70,-10.67,wh,3,440.730048
1,0330,nap_gellert_e,C622155141.0,dd,4.791268,92.44,1023.22,18.62,-10.59,wh,3,412.522497
2,9722,nap_gellert_e,C6305325.0,dd,4.905053,196.64,1866.71,17.58,-19.50,wh,4,649.552646
3,2400,nap_gellert_e,C63598134.0,dd,4.487733,72.81,673.45,8.64,-10.76,wh,2,282.611518
4,2400,nap_gellert_e,C6359838.0,dd,4.182375,83.03,673.45,8.24,-10.86,wh,2,255.707142
...,...,...,...,...,...,...,...,...,...,...,...,...
56,0085,nap_gellert_e,C78245617.0,dd,3.938365,774.54,8120.62,172.93,-219.05,wh,19,2846.921169
57,0085,nap_gellert_e,C78245945.0,dd,1.783998,3821.47,8685.16,239.14,-193.10,wh,18,4363.691776
58,1335,nap_gellert_e,C78257148.0,dd,5.509126,191.62,3407.49,99.13,-101.76,wh,4,1000.195321
59,1335,nap_gellert_e,C782571889.0,dd,5.169886,194.53,2724.58,87.49,-84.08,wh,4,807.428287


In [ ]:
if __name__ == "__main__":
    main(seed = 0, scale = 0)